# REDES NEURAIS ARTIFICIAIS

[Aula 3 de Deep Learning](http://bit.ly/dn-unb03) da Engenharia de Software da UnB

# Parte 1 - Pré-processamento dos Dados

### Importar as libs

In [76]:
import numpy as np
import pandas as pd
import tensorflow as tf

### Importar o dataset

In [77]:
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [78]:
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values
print("X >>\n",X)
print("y >>\n",y)

X >>
 [[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
y >>
 [1 0 1 ... 1 1 0]


### Transformando os dados categóricos


In [79]:
# Label Encoding the "Gender" column
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])
print(X)
# One Hot Encoding the "Geography" column
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]
[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


# Dividindo o dataset em conjunto de treinamento e conjunto de teste

In [80]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling

In [81]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [82]:
np.shape(X_train)

(8000, 12)

---
# Parte 2 -Vamos construir uma ANN!


### Initializing the ANN

In [83]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [84]:
ann.add(tf.keras.layers.Dense(units=10, activation='tanh'))

### Adding the second hidden layer

In [85]:
ann.add(tf.keras.layers.Dense(units=10, activation='tanh'))

### Adding the output layer

In [86]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the ANN

### Compiling the ANN

In [87]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the ANN on the Training set

In [88]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 1s 1ms/step - loss: 0.6259 - accuracy: 0.6508
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4298 - accuracy: 0.8193
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4214 - accuracy: 0.8196
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4088 - accuracy: 0.8220
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4035 - accuracy: 0.8229
Epoch 6/100
250/250 [==============================] - 0s 881us/step - loss: 0.3856 - accuracy: 0.8382
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3835 - accuracy: 0.8375
Epoch 8/100
250/250 [==============================] - 0s 961us/step - loss: 0.3800 - accuracy: 0.8422
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3754 - accuracy: 0.8451
Epoch 10/100
250/250 [==============================] - 0s 984us/step - loss: 0.3530 - 

250/250 [==============================] - 0s 1ms/step - loss: 0.3182 - accuracy: 0.8706
Epoch 81/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3232 - accuracy: 0.8628
Epoch 82/100
250/250 [==============================] - 0s 926us/step - loss: 0.3137 - accuracy: 0.8691
Epoch 83/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3120 - accuracy: 0.8744
Epoch 84/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3274 - accuracy: 0.8652
Epoch 85/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3182 - accuracy: 0.8716
Epoch 86/100
250/250 [==============================] - 0s 986us/step - loss: 0.3122 - accuracy: 0.8700
Epoch 87/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3076 - accuracy: 0.8744
Epoch 88/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3222 - accuracy: 0.8703
Epoch 89/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3223 - accura

# Parte 3 - Fazendo predições e avaliando o modelo

## Prevendo os resultados com o conjunto de testes

In [89]:
y_pred = ann.predict(X_test)
y_pred

array([[0.3532926 ],
       [0.12803122],
       [0.09227943],
       ...,
       [0.10461852],
       [0.21690011],
       [0.13060832]], dtype=float32)

In [90]:
y_pred = (y_pred > 0.5)
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

# Criando uma Confusion Matrix

In [91]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1505   90]
 [ 186  219]]


# FIM

> Professor Diego Dorgam  
> [@diegodorgam](https://twitter.com/diegodorgam)

### Nova Tarefa

In [92]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from tensorflow.keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import GridSearchCV

In [168]:
def classifier1():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(10, activation='tanh'))
    model.add(tf.keras.layers.Dense(10, activation='tanh'))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=["accuracy"])
    
    return model

def classifier2():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(10, activation='tanh'))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Dense(10, activation='tanh'))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=["accuracy"])

    return model

def improved_model(optimizer):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(10, activation='tanh'))
    model.add(tf.keras.layers.Dropout(0.1))
    model.add(tf.keras.layers.Dense(10, activation='tanh'))
    model.add(tf.keras.layers.Dropout(0.1))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer=optimizer, loss='binary_crossentropy',metrics=["accuracy"])

    return model

In [169]:
classifier_1 = KerasClassifier(build_fn = classifier1, batch_size = 10, epochs = 15)
classifier_2 = KerasClassifier(build_fn = classifier2, batch_size = 10, epochs = 15)

In [170]:
accu1 = cross_val_score(estimator = classifier_1, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean1 = accu1.mean()
var1 = accu1.std()

accu2 = cross_val_score(estimator = classifier_2, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean2 = accu2.mean()
var2 = accu2.std()

In [171]:
print(mean1, var1)
print(mean2, var2)

0.8594999969005584 0.009205975471611451
0.8376250028610229 0.007509374934298668


In [172]:
classifier = KerasClassifier(build_fn=improved_model)
params = {
    'batch_size': [10, 25],
    'optimizer': ['adam','rmsprop'],
    'epochs': [50,100]
}

grid_search = GridSearchCV(estimator=classifier,
                           scoring='accuracy',
                           param_grid=params,
                           cv=10)
grid_search = grid_search.fit(X_train, y_train)

Epoch 1/50
720/720 [==============================] - 2s 1ms/step - loss: 0.6023 - accuracy: 0.6797
Epoch 2/50
720/720 [==============================] - 1s 978us/step - loss: 0.4445 - accuracy: 0.8017
Epoch 3/50
720/720 [==============================] - 1s 1ms/step - loss: 0.4402 - accuracy: 0.8037
Epoch 4/50
720/720 [==============================] - 1s 1ms/step - loss: 0.4180 - accuracy: 0.8147
Epoch 5/50
720/720 [==============================] - 1s 2ms/step - loss: 0.4246 - accuracy: 0.8073
Epoch 6/50
720/720 [==============================] - 1s 1ms/step - loss: 0.4100 - accuracy: 0.8197
Epoch 7/50
720/720 [==============================] - 1s 995us/step - loss: 0.4029 - accuracy: 0.8211
Epoch 8/50
720/720 [==============================] - 1s 971us/step - loss: 0.3960 - accuracy: 0.8248
Epoch 9/50
720/720 [==============================] - 1s 1ms/step - loss: 0.3850 - accuracy: 0.8359
Epoch 10/50
720/720 [==============================] - 1s 897us/step - loss: 0.3870 - accuracy

720/720 [==============================] - 1s 888us/step - loss: 0.3707 - accuracy: 0.8487
Epoch 32/50
720/720 [==============================] - 1s 952us/step - loss: 0.3684 - accuracy: 0.8520
Epoch 33/50
720/720 [==============================] - 1s 877us/step - loss: 0.3589 - accuracy: 0.8534
Epoch 34/50
720/720 [==============================] - 1s 906us/step - loss: 0.3532 - accuracy: 0.8555
Epoch 35/50
720/720 [==============================] - 1s 970us/step - loss: 0.3621 - accuracy: 0.8503
Epoch 36/50
720/720 [==============================] - 1s 906us/step - loss: 0.3512 - accuracy: 0.8560
Epoch 37/50
720/720 [==============================] - 1s 900us/step - loss: 0.3409 - accuracy: 0.8621
Epoch 38/50
720/720 [==============================] - 1s 967us/step - loss: 0.3517 - accuracy: 0.8581
Epoch 39/50
720/720 [==============================] - 1s 924us/step - loss: 0.3502 - accuracy: 0.8554
Epoch 40/50
720/720 [==============================] - 1s 861us/step - loss: 0.3445 -

720/720 [==============================] - 1s 877us/step - loss: 0.3743 - accuracy: 0.8519
Epoch 11/50
720/720 [==============================] - 1s 946us/step - loss: 0.3910 - accuracy: 0.8303
Epoch 12/50
720/720 [==============================] - 1s 878us/step - loss: 0.3677 - accuracy: 0.8462
Epoch 13/50
720/720 [==============================] - 1s 875us/step - loss: 0.3632 - accuracy: 0.8474
Epoch 14/50
720/720 [==============================] - 1s 1ms/step - loss: 0.3687 - accuracy: 0.8411
Epoch 15/50
720/720 [==============================] - 1s 1ms/step - loss: 0.3668 - accuracy: 0.8527
Epoch 16/50
720/720 [==============================] - 1s 1ms/step - loss: 0.3593 - accuracy: 0.8531
Epoch 17/50
720/720 [==============================] - 1s 977us/step - loss: 0.3600 - accuracy: 0.8523
Epoch 18/50
720/720 [==============================] - 1s 888us/step - loss: 0.3516 - accuracy: 0.8537
Epoch 19/50
720/720 [==============================] - 1s 849us/step - loss: 0.3741 - accur

720/720 [==============================] - 1s 846us/step - loss: 0.3575 - accuracy: 0.8555
Epoch 40/50
720/720 [==============================] - 1s 904us/step - loss: 0.3428 - accuracy: 0.8657
Epoch 41/50
720/720 [==============================] - 1s 827us/step - loss: 0.3726 - accuracy: 0.8450
Epoch 42/50
720/720 [==============================] - 1s 818us/step - loss: 0.3626 - accuracy: 0.8514
Epoch 43/50
720/720 [==============================] - 1s 956us/step - loss: 0.3668 - accuracy: 0.8532
Epoch 44/50
720/720 [==============================] - 1s 800us/step - loss: 0.3529 - accuracy: 0.8596
Epoch 45/50
720/720 [==============================] - 1s 857us/step - loss: 0.3634 - accuracy: 0.8486
Epoch 46/50
720/720 [==============================] - 1s 1ms/step - loss: 0.3573 - accuracy: 0.8601
Epoch 47/50
720/720 [==============================] - 1s 857us/step - loss: 0.3559 - accuracy: 0.8553
Epoch 48/50
720/720 [==============================] - 1s 834us/step - loss: 0.3747 - a

720/720 [==============================] - 1s 2ms/step - loss: 0.3684 - accuracy: 0.8450
Epoch 19/50
720/720 [==============================] - 1s 1ms/step - loss: 0.3634 - accuracy: 0.8480
Epoch 20/50
720/720 [==============================] - 1s 1ms/step - loss: 0.3779 - accuracy: 0.8440
Epoch 21/50
720/720 [==============================] - 1s 996us/step - loss: 0.3640 - accuracy: 0.8457
Epoch 22/50
720/720 [==============================] - 1s 888us/step - loss: 0.3778 - accuracy: 0.8398
Epoch 23/50
720/720 [==============================] - 1s 832us/step - loss: 0.3394 - accuracy: 0.8642
Epoch 24/50
720/720 [==============================] - 1s 947us/step - loss: 0.3577 - accuracy: 0.8522
Epoch 25/50
720/720 [==============================] - 1s 835us/step - loss: 0.3689 - accuracy: 0.8484
Epoch 26/50
720/720 [==============================] - 1s 814us/step - loss: 0.3585 - accuracy: 0.8537
Epoch 27/50
720/720 [==============================] - 1s 942us/step - loss: 0.3603 - accur

...

288/288 [==============================] - 0s 883us/step - loss: 0.3462 - accuracy: 0.8582
Epoch 59/100
288/288 [==============================] - 0s 848us/step - loss: 0.3399 - accuracy: 0.8615
Epoch 60/100
288/288 [==============================] - 0s 789us/step - loss: 0.3730 - accuracy: 0.8436
Epoch 61/100
288/288 [==============================] - 0s 771us/step - loss: 0.3627 - accuracy: 0.8471
Epoch 62/100
288/288 [==============================] - 0s 782us/step - loss: 0.3543 - accuracy: 0.8550
Epoch 63/100
288/288 [==============================] - 0s 1ms/step - loss: 0.3559 - accuracy: 0.8581
Epoch 64/100
288/288 [==============================] - 0s 921us/step - loss: 0.3471 - accuracy: 0.8576
Epoch 65/100
288/288 [==============================] - 0s 820us/step - loss: 0.3570 - accuracy: 0.8557
Epoch 66/100
288/288 [==============================] - 0s 1ms/step - loss: 0.3614 - accuracy: 0.8476
Epoch 67/100
288/288 [==============================] - 0s 1ms/step - loss: 0.357

288/288 [==============================] - 0s 751us/step - loss: 0.3631 - accuracy: 0.8561
Epoch 38/100
288/288 [==============================] - 0s 879us/step - loss: 0.3492 - accuracy: 0.8563
Epoch 39/100
288/288 [==============================] - 0s 824us/step - loss: 0.3530 - accuracy: 0.8536
Epoch 40/100
288/288 [==============================] - 0s 834us/step - loss: 0.3630 - accuracy: 0.8463
Epoch 41/100
288/288 [==============================] - 0s 858us/step - loss: 0.3529 - accuracy: 0.8610
Epoch 42/100
288/288 [==============================] - 0s 817us/step - loss: 0.3630 - accuracy: 0.8477
Epoch 43/100
288/288 [==============================] - 0s 705us/step - loss: 0.3628 - accuracy: 0.8537
Epoch 44/100
288/288 [==============================] - 0s 782us/step - loss: 0.3645 - accuracy: 0.8555
Epoch 45/100
288/288 [==============================] - 0s 907us/step - loss: 0.3603 - accuracy: 0.8507
Epoch 46/100
288/288 [==============================] - 0s 716us/step - loss:

288/288 [==============================] - 0s 832us/step - loss: 0.3790 - accuracy: 0.8383
Epoch 17/100
288/288 [==============================] - 0s 1ms/step - loss: 0.3763 - accuracy: 0.8435
Epoch 18/100
288/288 [==============================] - 0s 1ms/step - loss: 0.3867 - accuracy: 0.8377
Epoch 19/100
288/288 [==============================] - 0s 855us/step - loss: 0.3811 - accuracy: 0.8380
Epoch 20/100
288/288 [==============================] - 0s 815us/step - loss: 0.3711 - accuracy: 0.8415
Epoch 21/100
288/288 [==============================] - 0s 865us/step - loss: 0.3729 - accuracy: 0.8419
Epoch 22/100
288/288 [==============================] - 0s 874us/step - loss: 0.3691 - accuracy: 0.8462
Epoch 23/100
288/288 [==============================] - 0s 816us/step - loss: 0.3786 - accuracy: 0.8421
Epoch 24/100
288/288 [==============================] - 0s 868us/step - loss: 0.3686 - accuracy: 0.8477
Epoch 25/100
288/288 [==============================] - 0s 956us/step - loss: 0.3

288/288 [==============================] - 0s 1ms/step - loss: 0.3506 - accuracy: 0.8560
Epoch 95/100
288/288 [==============================] - 0s 1ms/step - loss: 0.3353 - accuracy: 0.8635
Epoch 96/100
288/288 [==============================] - 1s 2ms/step - loss: 0.3579 - accuracy: 0.8570
Epoch 97/100
288/288 [==============================] - 0s 1ms/step - loss: 0.3573 - accuracy: 0.8528
Epoch 98/100
288/288 [==============================] - 0s 1ms/step - loss: 0.3501 - accuracy: 0.8604
Epoch 99/100
288/288 [==============================] - 0s 2ms/step - loss: 0.3602 - accuracy: 0.8537
Epoch 100/100
288/288 [==============================] - 0s 976us/step - loss: 0.3461 - accuracy: 0.8620
Epoch 1/100
800/800 [==============================] - 2s 1ms/step - loss: 0.5563 - accuracy: 0.7111
Epoch 2/100
800/800 [==============================] - 1s 966us/step - loss: 0.4359 - accuracy: 0.8094
Epoch 3/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4204 - accuracy

800/800 [==============================] - 1s 929us/step - loss: 0.3567 - accuracy: 0.8532
Epoch 74/100
800/800 [==============================] - 1s 877us/step - loss: 0.3431 - accuracy: 0.8662
Epoch 75/100
800/800 [==============================] - 1s 1ms/step - loss: 0.3546 - accuracy: 0.8558
Epoch 76/100
800/800 [==============================] - 1s 1ms/step - loss: 0.3571 - accuracy: 0.8545
Epoch 77/100
800/800 [==============================] - 1s 1ms/step - loss: 0.3523 - accuracy: 0.8546
Epoch 78/100
800/800 [==============================] - 1s 1ms/step - loss: 0.3528 - accuracy: 0.8525
Epoch 79/100
800/800 [==============================] - 1s 1ms/step - loss: 0.3687 - accuracy: 0.8530
Epoch 80/100
800/800 [==============================] - 1s 927us/step - loss: 0.3418 - accuracy: 0.8607
Epoch 81/100
800/800 [==============================] - 1s 765us/step - loss: 0.3492 - accuracy: 0.86430s - loss: 0
Epoch 82/100
800/800 [==============================] - 1s 917us/step - los

In [173]:
best_parameters = grid_search.best_params_
best_accu = grid_search.best_score_

In [174]:
print('Melhores parametrors: \n',best_parameters,'\nMelhor acurácia: \n', best_accu)

Melhores parametrors: 
 {'batch_size': 10, 'epochs': 100, 'optimizer': 'rmsprop'} 
Melhor acurácia: 
 0.8604999999999998
